# OPTaaS: Surrogate Visualization with Holoviews and Plotly

If you have the holoviews and plotly libraries installed it is also possible to get an interactive 3d view of
the surrogate predictions

In [ ]:
import numpy as np

from mindfoundry.optaas.client.goal import Goal
from mindfoundry.optaas.client.task import Task
from mindfoundry.optaas.client.parameter import FloatParameter, CategoricalParameter
from mindfoundry.optaas.client.viz import SurrogateViz

## Connect to OPTaaS using your API Key
from mindfoundry.optaas.client.client import OPTaaSClient
client = OPTaaSClient('https://optaas.mindfoundry.ai', '<Your OPTaaS API key>')

## Start the holoviews/plotly engine
import holoviews as hv
hv.extension('plotly')

## Create a simple task optimizing a function with 3 continuous and one categorical parameter

In [ ]:
task = client.create_task(
    title='Test',
    parameters=[
        FloatParameter('x', minimum=-2., maximum=2.),
        FloatParameter('y', minimum=-2., maximum=2.),
        FloatParameter('z', minimum=-2., maximum=2.),
        CategoricalParameter('c', values=["a", "b"]),
    ],
    goal=Goal.min,
)


## Define the scoring function
def scoring_function(x, y, z, c):
    rate = {"a": 1.5, "b": 0.75}[c] 
    score = np.sin(rate*x*3) + np.cos(rate*y*3) + np.sin(z*3) + {"a": -0.5, "b": 0}[c]
    return score


best_result = task.run(scoring_function, max_iterations=30)
print("Best Result:", best_result)

## Make a visualizer object and visualize the Surrogate Mean and Variance in 'x' and 'z'

In [ ]:
visualizer = SurrogateViz(client=client, task=task)
visualizer.plot_surrogate_mean_and_std("x","z", fig_width=800, fig_height=700, z_range=(-3,3.))

## We can also make a view with the uncertainties overlaid above and below the mean

In [ ]:
visualizer.plot_surrogate_with_uncertainties("x","z", fig_width=600, fig_height=700, z_range=(-4.,4.))